In [2]:
import pandas as pd

df = pd.DataFrame(pd.read_csv('banco.tsv', '\t'))
df

,CS_SEXO,NU_IDADE_N,CS_RACA,CS_ESCOL_N,SG_UF,ID_MN_RESI,CS_ZONA,SURTO_SG,NOSOCOMIAL,FEBRE,...,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TP_TES_AN,RES_AN,RES_IGG,RES_IGM,RES_IGA
0,M,79,1,2,SP,MAIRIPORA,1,2,2,1,...,9,9,9,9,1,1,5,9,9,4
1,M,3,4,9,MG,BELO HORIZONTE,1,2,2,1,...,2,2,2,2,6,1,4,4,4,4
2,M,78,1,2,RS,ALVORADA,1,2,2,2,...,2,2,2,2,6,2,1,4,4,4
3,M,58,4,2,SP,OSASCO,1,2,2,1,...,9,9,9,9,9,1,4,4,4,4
4,F,73,1,1,SP,OSASCO,1,1,2,1,...,2,2,2,2,9,1,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,M,72,1,2,RS,ALVORADA,1,2,2,2,...,2,2,2,2,9,2,2,4,4,4
1577,F,55,4,3,SP,OSASCO,1,1,2,2,...,2,2,2,2,9,1,4,4,4,4
1578,M,64,9,9,SP,ITANHAEM,1,2,2,2,...,2,2,2,2,1,1,4,4,4,4
1579,F,87,4,0,MG,TURMALINA,3,2,2,2,...,2,1,2,2,6,2,2,2,2,4


In [3]:
from mlxtend.frequent_patterns import apriori

class Apriori:

    threshold = 0.5
    df = None

    def __init__(self, df, threshold=None, transform_bol=False):
        """Apriori construtor 

        :param pandas.DataFrame df: transações do dataset
        :param float threshold: seta o threshold para o suporte minimo
        :return: intancia de apriori 
        :rtype: Apriori
        """

        self._validate_df(df)

        self.df = df
        if threshold is not None:
            self.threshold = threshold

        if transform_bol:
            self._transform_bol()

    def _validate_df(self, df=None):
        """valida se o dataframe existe. 

        :param pandas.DataFrame df: transações do dataset
        :return: 
        :rtype: void
        """

        if df is None:
            raise Exception("df precisa ser um pandas.DataDrame valido.")


    def _transform_bol(self):
        """transforma o dataframe(0 ou 1) para boleano (True ou False). 

        :return: 
        :rtype: void
        """

        for column in self.df.columns:
            self.df[column] = self.df[column].apply(lambda x: True if x == 1 else False)


    def _apriori(self, use_colnames=False, max_len=None, count=True):
        """chama a função apriori mlxtend.frequent_patterns . 

        :param bool use_colnames: sinaliza para usar as colunas no final do df.
        :param int max_len: tamanho maximo para os itemsets gerados.
        :param bool count: sinaliza a contagem do tamanho dos itemsets.
        :return: apriori DataFrame.
        :rtype: pandas.DataFrame
        """
    
        apriori_df = apriori(
                    self.df, 
                    min_support=self.threshold,
                    use_colnames=use_colnames, 
                    max_len=max_len
                )
        if count:
            apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))

        return apriori_df

    def run(self, use_colnames=False, max_len=None, count=True):
        """função de executar Apriori.

        :param bool use_colnames: sinaliza para usar as colunas no final do df.
        :param int max_len: tamanho maximo para os itemsets gerados.
        :param bool count: sinaliza a contagem do tamanho dos itemsets.
        :return: apriori DataFrame.
        :rtype: pandas.DataFrame
        """

        return self._apriori(
                        use_colnames=use_colnames,
                        max_len=max_len,
                        count=count
                    )

    def filter(self, apriori_df, length, threshold):
        """Filtra Apriori DataFrame por tamanho e threshold.

        :param pandas.DataFrame apriori_df: Apriori DataFrame.
        :param int length: tamanho dos itemsets requeridos.
        :param float threshold: minimo threshold requerido.
        :return: apriori DataFrame filtrado.
        :rtype:pandas.DataFrame
        """
        
        if 'length' not in apriori_df.columns:
            raise Exception("apriori_df has no length. Please run the Apriori with count=True.")

        return apriori_df[ (apriori_df['length'] == length) & (apriori_df['support'] >= threshold) ]

In [4]:
# executando o apriori
if 'ID' in df.columns: del df['ID'] # id nao é relevante apriori 

apriori_runner = Apriori(df, threshold=0.4, transform_bol=True)
apriori_df = apriori_runner.run(use_colnames=True)
apriori_df

,support,itemsets,length
0,0.440860,(CS_RACA),1
1,0.942441,(CS_ZONA),1
2,0.580645,(FEBRE),1
3,0.691335,(TOSSE),1
4,0.743201,(DISPNEIA),1
...,...,...,...
206,0.478178,"(SATURACAO, AMOSTRA, CS_ZONA, DESC_RESP, HOSPI...",5
207,0.401012,"(AMOSTRA, CS_ZONA, DESC_RESP, TP_TES_AN, HOSPI...",5
208,0.403542,"(TOSSE, AMOSTRA, DESC_RESP, DISPNEIA, HOSPITAL)",5
209,0.449715,"(SATURACAO, AMOSTRA, DESC_RESP, DISPNEIA, HOSP...",5


In [7]:
# Showing only pairs with support granter than 0.41
apriori_runner.filter(apriori_df, length=2, threshold=0.41)

,support,itemsets,length
13,0.426312,"(CS_RACA, CS_ZONA)",2
14,0.440860,"(CS_RACA, HOSPITAL)",2
15,0.440860,"(AMOSTRA, CS_RACA)",2
16,0.547755,"(FEBRE, CS_ZONA)",2
17,0.661607,"(TOSSE, CS_ZONA)",2
18,0.698925,"(DISPNEIA, CS_ZONA)",2
19,0.612903,"(DESC_RESP, CS_ZONA)",2
20,0.614801,"(SATURACAO, CS_ZONA)",2
21,0.440860,"(CARDIOPATI, CS_ZONA)",2
22,0.942441,"(CS_ZONA, HOSPITAL)",2
